In [4]:
%run "./Includes/Classroom-Setup"

### Tracking Experiments with MLflow

Over the course of the machine learning life cycle, data scientists test many different models from various libraries with different hyperparameters.  Tracking these various results poses an organizational challenge.  In brief, storing experiments, results, models, supplementary artifacts, and code creates significant challenges.

MLflow Tracking is one of the three main components of MLflow.  It is a logging API specific for machine learning and agnostic to libraries and environments that do the training.  It is organized around the concept of **runs**, which are executions of data science code.  Runs are aggregated into **experiments** where many runs can be a part of a given experiment and an MLflow server can host many experiments.

Each run can record the following information:<br><br>

- **Parameters:** Key-value pairs of input parameters such as the number of trees in a random forest model
- **Metrics:** Evaluation metrics such as RMSE or Area Under the ROC Curve
- **Artifacts:** Arbitrary output files in any format.  This can include images, pickled models, and data files
- **Source:** The code that originally ran the experiment

MLflow tracking also serves as a **model registry** so tracked models can easily be stored and, as necessary, deployed into production.

Experiments can be tracked using libraries in Python, R, and Java as well as by using the CLI and REST calls.  This course will use Python, though the majority of MLflow functionality is also exposed in these other APIs.

### Experiment Logging and UI

MLflow is an open source software project developed by Databricks available to developers regardless of which platform they are using.  Databricks hosts MLflow for you, which reduces deployment configuration and adds security benefits.  It is accessible on all Databricks workspaces in Azure and AWS.  It is not currently enabled on Community Edition

Databricks uses two different versions of its software.  The **Databricks Runtime** does not currently come with MLflow pre-installed.  if you are using Databricks runtime, uncomment the code below and run it to install `mlflow`.

In [9]:
# This installs MLflow for you only on Databricks Runtime
# NOTE: this code does not work with ML runtime (see below)

# dbutils.library.installPyPI("mlflow", "1.0.0")
# dbutils.library.restartPython()

The other version of Databricks is the **Machine Learning Runtime,** which you'll notice has `ML` next to the cluster type.  **Machine Learning Runtime has `mlflow` pre-installed starting at version 5.5.**  If you have runtime **5.5 ML or later** available to you, you do not need to install `mlflow`.

In [11]:
!pip install mlflow

Import a dataset of Airbnb listings and featurize the data.  We'll use this to train a model.

In [13]:
import pandas as pd

df = pd.read_csv("/dbfs/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv")

Perform a train/test split.

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(["price"], axis=1), df[["price"]].values.ravel(), random_state=42)

**Navigate to the MLflow UI by clicking on the `Runs` button on the top of the screen.**

Every Python notebook in a Azure Databricks Workspace has its own experiment. When you use MLflow in a notebook, it records runs in the notebook experiment. A notebook experiment shares the same name and ID as its corresponding notebook.

Log a basic experiment by doing the following:<br><br>

1. Start an experiment using `mlflow.start_run()` and passing it a name for the run
2. Train your model
3. Log the model using `mlflow.sklearn.log_model()`
4. Log the model error using `mlflow.log_metric()`
5. Print out the run id using `run.info.run_uuid`

In [18]:
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

with mlflow.start_run(run_name="Basic RF Experiment") as run:
  # Create model, train it, and create predictions
  rf = RandomForestRegressor()
  rf.fit(X_train, y_train)
  predictions = rf.predict(X_test)
  
  # Log model
  mlflow.sklearn.log_model(rf, "random-forest-model")
  
  # Create metrics
  mse = mean_squared_error(y_test, predictions)
  print("  mse: {}".format(mse))
  
  # Log metrics
  mlflow.log_metric("mse", mse)
  
  runID = run.info.run_uuid
  experimentID = run.info.experiment_id
  
  print("Inside MLflow Run with run_id {} and experiment_id {}".format(runID, experimentID))

Thu Apr 16 07:47:54 2020 py4j imported
/databricks/python/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
 "10 in version 0.20 to 100 in 0.22.", FutureWarning)
 mse: 6998.4432115548
Inside MLflow Run with run_id aef808c7a24547e18e732577a4f03b51 and experiment_id 3227357976395900

Examine the results in the UI.  Look for the following:<br><br>

1. The `Experiment ID`
2. The artifact location.  This is where the artifacts are stored in DBFS, which is backed by an Amazon S3 bucket
3. The time the run was executed.  **Click this timestamp to see more information on the run.**
4. The code that executed the run.

After clicking on the time of the run, take a look at the following:<br><br>

1. The Run ID will match what we printed above
2. The model that we saved, included a picked version of the model as well as the Conda environment and the `MLmodel` file, which will be discussed in the next lesson.

Now take a look at the directory structure backing this experiment.  This allows you to retrieve artifacts.

In [22]:
from mlflow.tracking import MlflowClient

artifactURL = MlflowClient().get_experiment(experimentID).artifact_location
dbutils.fs.ls(artifactURL)

Out[20]: [FileInfo(path='dbfs:/databricks/mlflow/3227357976395900/aef808c7a24547e18e732577a4f03b51/', name='aef808c7a24547e18e732577a4f03b51/', size=0)]

Take a look at the contents of `random-forest-model`, which match what we see in the UI.

In [24]:
modelURL = artifactURL + "/" + runID + "/artifacts/random-forest-model"
dbutils.fs.ls(modelURL)

Out[21]: [FileInfo(path='dbfs:/databricks/mlflow/3227357976395900/aef808c7a24547e18e732577a4f03b51/artifacts/random-forest-model/MLmodel', name='MLmodel', size=357),
 FileInfo(path='dbfs:/databricks/mlflow/3227357976395900/aef808c7a24547e18e732577a4f03b51/artifacts/random-forest-model/conda.yaml', name='conda.yaml', size=130),
 FileInfo(path='dbfs:/databricks/mlflow/3227357976395900/aef808c7a24547e18e732577a4f03b51/artifacts/random-forest-model/model.pkl', name='model.pkl', size=2532908)]

### Parameters, Metrics, and Artifacts

But wait, there's more!  In the last example, you logged the run name, an evaluation metric, and your model itself as an artifact.  Now let's log parameters, multiple metrics, and other artifacts including the feature importances.

First, create a function to perform this.

To log artifacts, we have to save them somewhere before MLflow can log them.  This code accomplishes that by using a temporary file that it then deletes.

In [26]:
def log_rf(experimentID, run_name, params, X_train, X_test, y_train, y_test):
  import os
  import matplotlib.pyplot as plt
  import mlflow.sklearn
  import seaborn as sns
  from sklearn.ensemble import RandomForestRegressor
  from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
  import tempfile

  with mlflow.start_run(experiment_id=experimentID, run_name=run_name) as run:
    # Create model, train it, and create predictions
    rf = RandomForestRegressor(**params)
    rf.fit(X_train, y_train)
    predictions = rf.predict(X_test)

    # Log model
    mlflow.sklearn.log_model(rf, "random-forest-model")

    # Log params
    [mlflow.log_param(param, value) for param, value in params.items()]

    # Create metrics
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    print("  mse: {}".format(mse))
    print("  mae: {}".format(mae))
    print("  R2: {}".format(r2))

    # Log metrics
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("mae", mae)  
    mlflow.log_metric("r2", r2)  
    
    # Create feature importance
    importance = pd.DataFrame(list(zip(df.columns, rf.feature_importances_)), 
                                columns=["Feature", "Importance"]
                              ).sort_values("Importance", ascending=False)
    
    # Log importances using a temporary file
    temp = tempfile.NamedTemporaryFile(prefix="feature-importance-", suffix=".csv")
    temp_name = temp.name
    try:
      importance.to_csv(temp_name, index=False)
      mlflow.log_artifact(temp_name, "feature-importance.csv")
    finally:
      temp.close() # Delete the temp file
    
    # Create plot
    fig, ax = plt.subplots()

    sns.residplot(predictions, y_test, lowess=True)
    plt.xlabel("Predicted values for Price ($)")
    plt.ylabel("Residual")
    plt.title("Residual Plot")

    # Log residuals using a temporary file
    temp = tempfile.NamedTemporaryFile(prefix="residuals-", suffix=".png")
    temp_name = temp.name
    try:
      fig.savefig(temp_name)
      mlflow.log_artifact(temp_name, "residuals.png")
    finally:
      temp.close() # Delete the temp file
      
    display(fig)
    return run.info.run_uuid

Run with new parameters.

In [28]:
params = {
  "n_estimators": 100,
  "max_depth": 5,
  "random_state": 42
}

log_rf(experimentID, "Second Run", params, X_train, X_test, y_train, y_test)

Check the UI to see how this appears.  Take a look at the artifact to see where the plot was saved.

Now, run a third run.

In [30]:
params_1000_trees = {
  "n_estimators": 1000,
  "max_depth": 10,
  "random_state": 42
}

log_rf(experimentID, "Third Run", params_1000_trees, X_train, X_test, y_train, y_test)

### Querying Past Runs

You can query past runs programatically in order to use this data back in Python.  The pathway to doing this is an `MlflowClient` object. 

In [32]:
from  mlflow.tracking import MlflowClient

client = MlflowClient()

Now list all the runs for your experiment using `.list_run_infos()`, which takes your `experiment_id` as a parameter.

In [34]:
client.list_run_infos(experimentID)

Out[26]: [<RunInfo: artifact_uri='dbfs:/databricks/mlflow/3227357976395900/5a9199cd9e8d4387b3345de0af61871e/artifacts', end_time=1587023519377, experiment_id='3227357976395900', lifecycle_stage='active', run_id='5a9199cd9e8d4387b3345de0af61871e', run_uuid='5a9199cd9e8d4387b3345de0af61871e', start_time=1587023490410, status='FINISHED', user_id=''>,
 <RunInfo: artifact_uri='dbfs:/databricks/mlflow/3227357976395900/70204172cc4d4c59a38035f4e03cf96b/artifacts', end_time=1587023476035, experiment_id='3227357976395900', lifecycle_stage='active', run_id='70204172cc4d4c59a38035f4e03cf96b', run_uuid='70204172cc4d4c59a38035f4e03cf96b', start_time=1587023470895, status='FINISHED', user_id=''>,
 <RunInfo: artifact_uri='dbfs:/databricks/mlflow/3227357976395900/aef808c7a24547e18e732577a4f03b51/artifacts', end_time=1587023278520, experiment_id='3227357976395900', lifecycle_stage='active', run_id='aef808c7a24547e18e732577a4f03b51', run_uuid='aef808c7a24547e18e732577a4f03b51', start_time=1587023274734, status='FINISHED', user_id=''>]

Pull out a few fields and create a pandas DataFrame with it.

In [36]:
runs = pd.DataFrame([(run.run_uuid, run.start_time, run.artifact_uri) for run in client.list_run_infos(experimentID)])
runs.columns = ["run_uuid", "start_time", "artifact_uri"]

display(runs)

run_uuid,start_time,artifact_uri
5a9199cd9e8d4387b3345de0af61871e,1587023490410,dbfs:/databricks/mlflow/3227357976395900/5a9199cd9e8d4387b3345de0af61871e/artifacts
70204172cc4d4c59a38035f4e03cf96b,1587023470895,dbfs:/databricks/mlflow/3227357976395900/70204172cc4d4c59a38035f4e03cf96b/artifacts
aef808c7a24547e18e732577a4f03b51,1587023274734,dbfs:/databricks/mlflow/3227357976395900/aef808c7a24547e18e732577a4f03b51/artifacts


Pull the last run and take a look at the associated artifacts.

In [38]:
last_run = runs.sort_values("start_time", ascending=False).iloc[0]

dbutils.fs.ls(last_run["artifact_uri"]+"/random-forest-model/")

Out[28]: [FileInfo(path='dbfs:/databricks/mlflow/3227357976395900/5a9199cd9e8d4387b3345de0af61871e/artifacts/random-forest-model/MLmodel', name='MLmodel', size=357),
 FileInfo(path='dbfs:/databricks/mlflow/3227357976395900/5a9199cd9e8d4387b3345de0af61871e/artifacts/random-forest-model/conda.yaml', name='conda.yaml', size=130),
 FileInfo(path='dbfs:/databricks/mlflow/3227357976395900/5a9199cd9e8d4387b3345de0af61871e/artifacts/random-forest-model/model.pkl', name='model.pkl', size=46665877)]

Return the evaluation metrics for the last run.

In [40]:
client.get_run(last_run.run_uuid).data.metrics

Out[29]: {'mae': 48.82679545743182, 'mse': 6013.141341974693, 'r2': 0.7063859458544959}

Reload the model and take a look at the feature importance.

In [42]:
dbutils.fs.ls(last_run.artifact_uri)

Out[30]: [FileInfo(path='dbfs:/databricks/mlflow/3227357976395900/5a9199cd9e8d4387b3345de0af61871e/artifacts/feature-importance.csv/', name='feature-importance.csv/', size=0),
 FileInfo(path='dbfs:/databricks/mlflow/3227357976395900/5a9199cd9e8d4387b3345de0af61871e/artifacts/random-forest-model/', name='random-forest-model/', size=0),
 FileInfo(path='dbfs:/databricks/mlflow/3227357976395900/5a9199cd9e8d4387b3345de0af61871e/artifacts/residuals.png/', name='residuals.png/', size=0)]

In [43]:
import mlflow.sklearn

model = mlflow.sklearn.load_model(last_run.artifact_uri + "/random-forest-model/")
model.feature_importances_

Out[31]: array([0.02066125, 0.01968234, 0.01714745, 0.10652883, 0.0466081 ,
 0.0197694 , 0.04940956, 0.05043625, 0.09587796, 0.45534329,
 0.01225107, 0.00644477, 0.01970861, 0.03186636, 0.0288097 ,
 0.00256674, 0.00629972, 0.00134757, 0.0006453 , 0.00505837,
 0.00353737])

## Review
**Question:** What can MLflow Tracking log?  
**Answer:** MLflow can log the following:
- **Parameters:** inputs to a model
- **Metrics:** the performance of the model
- **Artifacts:** any object including data, models, and images
- **Source:** the original code, including the commit hash if linked to git

**Question:** How do you log experiments?  
**Answer:** Experiments are logged by first creating a run and using the logging methods on that run object (e.g. `run.log_param("MSE", .2)`).

**Question:** Where do logged artifacts get saved?  
**Answer:** Logged artifacts are saved in a directory of your choosing.  On Databricks, this would be DBFS, or the Databricks File System, which backed by a blob store.

**Question:** How can I query past runs?  
**Answer:** This can be done using an `MlflowClient` object.  This allows you do everything you can within the UI programatically so you never have to step outside of your programming environment.

In [46]:
%run "./Includes/Classroom-Cleanup"